In [4]:
# === 0. Directorio de trabajo ===

import os
from pathlib import Path

# Detectamos la ruta donde se encuentra este notebook
notebook_dir = Path().resolve()
print(f"Directorio actual del notebook:\n{notebook_dir}\n")

# Ajustamos el directorio de trabajo a esa misma ruta
os.chdir(notebook_dir)

# Confirmamos el cambio
print(f"Nuevo directorio de trabajo:\n{os.getcwd()}")

Directorio actual del notebook:
C:\Users\econz\OneDrive\Documentos\Lecturas libros\Cursos UBA\Programación\Grupo4_UBA_2025\TP3

Nuevo directorio de trabajo:
C:\Users\econz\OneDrive\Documentos\Lecturas libros\Cursos UBA\Programación\Grupo4_UBA_2025\TP3


In [6]:
# === Librerías ===
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import requests, zipfile, io
import statsmodels.api as sm

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures 
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from scipy import stats

# Configuración general
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:,.2f}'.format
sns.set(style="whitegrid", palette="deep")

In [7]:
# === Cargar base del TP2 ===

import os
import pandas as pd
from pathlib import Path

# Directorio donde está guardado este notebook y las bases
DATOS = Path().resolve()

# Archivo eph_base.csv dentro del TP3
ruta = DATOS / "eph_base.csv"

# Cargar base respondieron
eph = pd.read_csv(ruta)

# Verificación rápida
print("Base cargada correctamente.")
print("Filas y columnas:", eph.shape)
print("\nVista previa:")
display(eph.head(3))

Base cargada correctamente.
Filas y columnas: (7340, 37)

Vista previa:


,componente,ano4,codusu,nro_hogar,Sexo,ch06,ch07,ch08,nivel_ed,estado,cat_ocup,cat_inac,itf,ipcf,pp07h,pp03c,p47t,p21,pp03g,Edad,adulto_equiv,ad_equiv_hogar,ingreso_necesario,pobre,edad2,ch10,ch12,ch13,ch14,educ,itf_2025,linea_pobreza,pp3e_tot,pp3f_tot,ch03,horastrab,ix_tot
0,1.00,2005,125814,1,Varón,46,Casado,Obra social (incluye PAMI),Secundaria Incompleta,Ocupado,Obrero o empleado,NaN,"2,400.00",480.00,Sí,...un sólo empleo/ocupación/actividad?,"2,400.00","2,400.00",No,46 a 60 años,1.00,3.93,805.93,0,2116,2,4,2,2.00,10.00,"2,518,778.11","845,811.04",48.00,0.00,1,48.00,5.00
1,2.00,2005,125814,1,Mujer,32,Casado,Obra social (incluye PAMI),Secundaria Incompleta,Inactivo,NaN,Ama de casa,"2,400.00",480.00,NaN,NaN,0.00,0.00,NaN,30 a 45 años,0.77,3.93,805.93,0,1024,2,4,2,2.00,10.00,"2,518,778.11","845,811.04",0.00,0.00,2,0.00,5.00
2,3.00,2005,125814,1,Varón,14,Soltero,Obra social (incluye PAMI),Primaria Completa,Inactivo,NaN,Estudiante,"2,400.00",480.00,NaN,NaN,0.00,0.00,NaN,14 años,0.96,3.93,805.93,0,196,1,3,2,7.00,8.00,"2,518,778.11","845,811.04",0.00,0.00,3,0.00,5.00


In [8]:
# Hay duplicados?
print("Duplicados:", eph.duplicated().sum())

# Hay valores faltantes?
print("\n Missings:\n", eph.isnull().sum()) # conteo
#print(auto.isnull().mean() * 100) # como porcentaje

# No hay duplicados ni missing values

Duplicados: 0

 Missings:
 componente              0
ano4                    0
codusu                  0
nro_hogar               0
Sexo                    0
ch06                    0
ch07                    0
ch08                   17
nivel_ed                0
estado                407
cat_ocup             4109
cat_inac             3278
itf                     0
ipcf                    0
pp07h                4796
pp03c                4526
p47t                    0
p21                     0
pp03g                4264
Edad                    0
adulto_equiv            0
ad_equiv_hogar          0
ingreso_necesario       0
pobre                   0
edad2                   0
ch10                    0
ch12                    0
ch13                    0
ch14                 3465
educ                  142
itf_2025                0
linea_pobreza           0
pp3e_tot             2333
pp3f_tot             2333
ch03                    0
horastrab              30
ix_tot                  0
dtype: int6

In [9]:
### Arreglamos un filtro adicional que encontramos que nos había faltado en la limpieza del TP1


#Estado
def clasificar_estado(estado, ch06):
    if pd.isna(estado) and ch06 < 10:
        return "Menor de 10 años"
    return estado

eph["estado"] = eph.apply(
    lambda row: clasificar_estado(row["estado"], row["ch06"]),
    axis=1)


#cat_ocup
def completar_cat_ocup(cat_ocup, estado):
    if pd.isna(cat_ocup):
        if estado in ["Inactivo", "Menor de 10 años", "Desocupado"]:
            return "No ocupado"
    return cat_ocup

eph["cat_ocup"] = eph.apply(
    lambda row: completar_cat_ocup(row["cat_ocup"], row["estado"]),
    axis=1
)



#Inactivo
def clasificar_estado(cat_inac, estado):
    if pd.isna(cat_inac) and estado == "Ocupado":
        return "Ocupado"
    if pd.isna(cat_inac) and estado == "Desocupado":
        return "Desocupado"
    return cat_inac

eph["cat_inac"] = eph.apply(
    lambda row: clasificar_estado(row["cat_inac"], row["estado"]),
    axis=1)


In [11]:

#dropeamos las filas con nan que son inconsistentes. son muy pocas observacions no afectan los resultados.

eph[["estado","ch08","educ","horastrab"]].isna().sum()

estado         1
ch08          17
educ         142
horastrab     30
dtype: int64

In [13]:
eph = eph.dropna(
    subset=["estado", "ch08", "educ","horastrab"]
)

In [14]:
# Hay valores faltantes?
print("\n Missings:\n", eph.isnull().sum()) # conteo

#Solo quedan nan variables numericas asocidas a personas con trabajo


 Missings:
 componente              0
ano4                    0
codusu                  0
nro_hogar               0
Sexo                    0
ch06                    0
ch07                    0
ch08                    0
nivel_ed                0
estado                  0
cat_ocup                0
cat_inac                0
itf                     0
ipcf                    0
pp07h                4642
pp03c                4389
p47t                    0
p21                     0
pp03g                4127
Edad                    0
adulto_equiv            0
ad_equiv_hogar          0
ingreso_necesario       0
pobre                   0
edad2                   0
ch10                    0
ch12                    0
ch13                    0
ch14                 3439
educ                    0
itf_2025                0
linea_pobreza           0
pp3e_tot             2304
pp3f_tot             2304
ch03                    0
horastrab               0
ix_tot                  0
dtype: int64


# A. Enfoque de validación

In [15]:
#Simplificamos variables categóricas con muchas categorías de manera innecesaria y categorías innecesarias para predicción y nos quedamos con las de interés

eph = eph[
    eph["ch08"] != "Ns./Nr."
]

eph = eph[
    eph["nivel_ed"] != "Ns./Nr."
]

eph = eph[
    eph["cat_inac"] != "Otros"
]


eph["ch10"] = eph["ch10"].astype("category")

eph["cobertura_medica"] = eph["ch08"].apply(
    lambda x: "sin_cobertura_medica" if x == "No paga ni le descuentan" else "con_cobertura_medica"
)



cols = [
    "ano4",
    "componente",
    "codusu",
    "nro_hogar",
    "Sexo",        # corregido
    "ch06",
    "nivel_ed",
    "estado",
    "cat_ocup",    # corregido
    "cat_inac",    # interpretando tu pedido
    "pobre",
    "ch10"
]

eph_variables_seleccionadas = eph[cols]


In [16]:
#Vemos cuales variables eran categóricas

# Variables e información
#print(auto.dtypes)
print(eph_variables_seleccionadas.info())


<class 'pandas.core.frame.DataFrame'>
Index: 7024 entries, 0 to 7339
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   ano4        7024 non-null   int64   
 1   componente  7024 non-null   float64 
 2   codusu      7024 non-null   object  
 3   nro_hogar   7024 non-null   int64   
 4   Sexo        7024 non-null   object  
 5   ch06        7024 non-null   int64   
 6   nivel_ed    7024 non-null   object  
 7   estado      7024 non-null   object  
 8   cat_ocup    7024 non-null   object  
 9   cat_inac    7024 non-null   object  
 10  pobre       7024 non-null   int64   
 11  ch10        7024 non-null   category
dtypes: category(1), float64(1), int64(4), object(6)
memory usage: 665.6+ KB
None


In [17]:
#Creamos dummies para las variables categóricas:

eph_con_dummies = pd.get_dummies(
    eph_variables_seleccionadas.drop(columns=["codusu"]),
    drop_first=True,
    dtype=int
)

# Si querés volver a agregar codusu
eph_con_dummies["codusu"] = eph_variables_seleccionadas["codusu"]

eph_con_dummies.head()

,ano4,componente,nro_hogar,ch06,pobre,Sexo_Varón,nivel_ed_Primaria Incompleta (incluye educación especial),nivel_ed_Secundaria Completa,nivel_ed_Secundaria Incompleta,nivel_ed_Sin instrucción,nivel_ed_Superior Universitaria Completa,nivel_ed_Superior Universitaria Incompleta,estado_Inactivo,estado_Menor de 10 años,estado_Ocupado,cat_ocup_No ocupado,cat_ocup_Obrero o empleado,cat_ocup_Patrón,cat_ocup_Trabajador familiar sin remuneración,cat_inac_Desocupado,cat_inac_Discapacitado,cat_inac_Estudiante,cat_inac_Jubilado/pensionado,cat_inac_Menor de 6 años,cat_inac_Ocupado,cat_inac_Rentista,ch10_1,ch10_2,ch10_3,codusu
0,2005,1.00,1,46,0,1,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,1,0,125814
1,2005,2.00,1,32,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,125814
2,2005,3.00,1,14,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,125814
3,2005,4.00,1,9,0,1,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,125814
4,2005,5.00,1,3,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,125814


In [19]:
# Guardo los vectores de variable dependiente y de variable independiente respectivamente:
y = eph_con_dummies['pobre']
X = eph_con_dummies.drop(columns=["pobre", "componente", "nro_hogar", "codusu"])

# Agrego columna de 1s para el intercepto
X = sm.add_constant(X)

### 1. Test de medias

In [20]:
#Test de medias

resultados = []
for col in X.columns:
    x = X[col].values.reshape(-1, 1)
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=444, stratify=y)
    t_x = stats.ttest_ind(x_train.flatten(), x_test.flatten())
    resultados.append({
        'Variable': col,
        'N train': x_train.shape[0],
        'Mean train': x_train.mean(),
        'sd train': x_train.std(),
        'N test': x_test.shape[0],
        'Mean test': x_test.mean(),
        'sd test': x_test.std(),
        't-test': t_x.statistic,
        'p-value': t_x.pvalue
    })

estadisticos = pd.DataFrame(resultados)
estadisticos.to_excel('dif_medias_multiple.xlsx', index=False)
estadisticos

C:\ProgramData\anaconda3\Lib\site-packages\scipy\stats\_axis_nan_policy.py:586: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  res = hypotest_fun_out(*samples, **kwds)


,Variable,N train,Mean train,sd train,N test,Mean test,sd test,t-test,p-value
0,const,4916,1.00,0.00,2108,1.00,0.00,NaN,NaN
1,ano4,4916,"2,016.40",9.90,2108,"2,016.52",9.88,-0.44,0.66
2,ch06,4916,34.75,21.52,2108,34.12,21.48,1.13,0.26
3,Sexo_Varón,4916,0.49,0.50,2108,0.48,0.50,1.31,0.19
4,nivel_ed_Primaria Incompleta (incluye educació...,4916,0.18,0.38,2108,0.17,0.38,0.44,0.66
5,nivel_ed_Secundaria Completa,4916,0.20,0.40,2108,0.21,0.41,-1.15,0.25
6,nivel_ed_Secundaria Incompleta,4916,0.22,0.41,2108,0.23,0.42,-0.48,0.63
7,nivel_ed_Sin instrucción,4916,0.06,0.24,2108,0.07,0.25,-1.39,0.16
8,nivel_ed_Superior Universitaria Completa,4916,0.11,0.31,2108,0.11,0.31,-0.02,0.99
9,nivel_ed_Superior Universitaria Incompleta,4916,0.10,0.30,2108,0.09,0.29,1.00,0.32


In [31]:
# === Test de medias según consigna del TP3 ===

from scipy import stats

# Train-test único y válido
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=444, stratify=y
)

resultados_validos = []

for col in X.columns:
    t_x = stats.ttest_ind(
        X_train[col],
        X_test[col],
        equal_var=False,
        nan_policy='omit'
    )
    
    resultados_validos.append({
        'Variable': col,
        'Mean_train': X_train[col].mean(),
        'Mean_test': X_test[col].mean(),
        'SD_train': X_train[col].std(),
        'SD_test': X_test[col].std(),
        't_stat': t_x.statistic,
        'p_value': t_x.pvalue
    })

tabla_test_medias = pd.DataFrame(resultados_validos)
tabla_test_medias


C:\ProgramData\anaconda3\Lib\site-packages\scipy\stats\_axis_nan_policy.py:586: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  res = hypotest_fun_out(*samples, **kwds)


,Variable,Mean_train,Mean_test,SD_train,SD_test,t_stat,p_value
0,const,1.00,1.00,0.00,0.00,NaN,NaN
1,ano4,"2,016.40","2,016.52",9.90,9.89,-0.44,0.66
2,ch06,34.75,34.12,21.52,21.48,1.13,0.26
3,Sexo_Varón,0.49,0.48,0.50,0.50,1.31,0.19
4,nivel_ed_Primaria Incompleta (incluye educació...,0.18,0.17,0.38,0.38,0.44,0.66
5,nivel_ed_Secundaria Completa,0.20,0.21,0.40,0.41,-1.14,0.26
6,nivel_ed_Secundaria Incompleta,0.22,0.23,0.42,0.42,-0.48,0.63
7,nivel_ed_Sin instrucción,0.06,0.07,0.24,0.25,-1.36,0.18
8,nivel_ed_Superior Universitaria Completa,0.11,0.11,0.31,0.31,-0.02,0.99
9,nivel_ed_Superior Universitaria Incompleta,0.10,0.09,0.30,0.29,1.01,0.31


### 2. Separación en respondieron_2005 y respondieron_2025

In [23]:
# Separar en dos dataframes según el año
respondieron_2005 = eph[eph['ano4'] == 2005].copy()
respondieron_2005.sample(3)

,componente,ano4,codusu,nro_hogar,Sexo,ch06,ch07,ch08,nivel_ed,estado,cat_ocup,cat_inac,itf,ipcf,pp07h,pp03c,p47t,p21,pp03g,Edad,adulto_equiv,ad_equiv_hogar,ingreso_necesario,pobre,edad2,ch10,ch12,ch13,ch14,educ,itf_2025,linea_pobreza,pp3e_tot,pp3f_tot,ch03,horastrab,ix_tot,cobertura_medica
399,2.00,2005,205000,1,Varón,50,Casado,Obra social (incluye PAMI),Primaria Completa,Ocupado,Obrero o empleado,Ocupado,"1,520.00",304.00,Sí,...un sólo empleo/ocupación/actividad?,820.00,820.00,No,46 a 60 años,1.00,4.25,871.55,0,2500,2,1,1,NaN,1.00,"1,595,226.14","914,681.15",46.00,0.00,9,46.00,5.00,con_cobertura_medica
2825,1.00,2005,220143,1,Mujer,54,Separado o divorciado,No paga ni le descuentan,Secundaria Completa,Ocupado,Obrero o empleado,Ocupado,"3,580.00","1,193.33",No,NaN,300.00,300.00,No,46 a 60 años,0.76,2.54,520.88,0,2916,2,4,1,NaN,13.00,"3,757,177.35","546,656.50",0.00,0.00,1,0.00,3.00,sin_cobertura_medica
2752,2.00,2005,202371,1,Mujer,45,Unido,Obra social (incluye PAMI),Superior Universitaria Incompleta,Ocupado,Obrero o empleado,Ocupado,"4,590.00","2,295.00",Sí,...un sólo empleo/ocupación/actividad?,"2,490.00","1,800.00",No,30 a 45 años,0.77,1.77,362.97,0,2025,1,6,2,0.00,13.00,"4,817,163.14","380,937.80",35.00,0.00,2,35.00,2.00,con_cobertura_medica


In [25]:
respondieron_2025 = eph[eph['ano4'] == 2025].copy()
respondieron_2025.sample(3)

,componente,ano4,codusu,nro_hogar,Sexo,ch06,ch07,ch08,nivel_ed,estado,cat_ocup,cat_inac,itf,ipcf,pp07h,pp03c,p47t,p21,pp03g,Edad,adulto_equiv,ad_equiv_hogar,ingreso_necesario,pobre,edad2,ch10,ch12,ch13,ch14,educ,itf_2025,linea_pobreza,pp3e_tot,pp3f_tot,ch03,horastrab,ix_tot,cobertura_medica
3568,3.00,2025,TQRMNOPVUHLOKUCDEGNFJ00853247,1,Mujer,17,Soltero,Obra social (incluye PAMI),Secundaria Incompleta,Inactivo,No ocupado,Estudiante,"1,000,000.00","333,333.33",NaN,NaN,0.00,0.00,NaN,17 años,0.77,2.54,"927,549.58",0,289,1,4,2,5.00,13.00,"1,000,000.00","927,549.58",NaN,NaN,3,0.00,3.00,con_cobertura_medica
3565,2.00,2025,TQRMNOPUPHKOKPCDEGNFJ00857719,1,Varón,73,Casado,Obra social (incluye PAMI),Primaria Completa,Ocupado,Obrero o empleado,Ocupado,"3,225,000.00","1,612,500.00",Sí,...un sólo empleo/ocupación/actividad?,"1,800,000.00","1,200,000.00",No,61 a 75 años,0.83,1.50,"547,765.50",0,5329,2,2,1,NaN,8.00,"3,225,000.00","547,765.50",35.00,0.00,2,35.00,2.00,con_cobertura_medica
6873,2.00,2025,TQRMNORVYHLMKMCDEOJAH00859128,1,Mujer,54,Unido,Obra social (incluye PAMI),Superior Universitaria Completa,Ocupado,Obrero o empleado,Ocupado,"2,180,000.00","1,090,000.00",Sí,...un sólo empleo/ocupación/actividad?,"980,000.00","980,000.00",No,46 a 60 años,0.76,1.76,"642,711.52",0,2916,2,6,1,NaN,16.00,"2,180,000.00","642,711.52",40.00,0.00,2,40.00,2.00,con_cobertura_medica


### Definir X_2025 y y_2025 (solo para el año 2025)

In [26]:
# === Matrices X e y específicas para 2025 ===

X_2025 = X.loc[respondieron_2025.index].copy()
y_2025 = y.loc[respondieron_2025.index].copy()

print("X_2025:", X_2025.shape)
print("y_2025:", y_2025.shape)


X_2025: (4017, 27)
y_2025: (4017,)


### Hacer train-test split SOLO para 2025

In [27]:
from sklearn.model_selection import train_test_split

X_train_2025, X_test_2025, y_train_2025, y_test_2025 = train_test_split(
    X_2025,
    y_2025,
    test_size=0.3,
    random_state=444,
    stratify=y_2025
)

print("Tamaño train:", X_train_2025.shape)
print("Tamaño test:", X_test_2025.shape)

Tamaño train: (2811, 27)
Tamaño test: (1206, 27)


### Tabla de diferencias de medias PERO AHORA SOLO PARA 2025

In [28]:
# === Test de medias para 2025 ===

resultados_2025 = []

for col in X_2025.columns:
    t_x = stats.ttest_ind(
        X_train_2025[col],
        X_test_2025[col],
        equal_var=False,
        nan_policy='omit'
    )
    
    resultados_2025.append({
        'Variable': col,
        'Mean_train': X_train_2025[col].mean(),
        'Mean_test': X_test_2025[col].mean(),
        'SD_train': X_train_2025[col].std(),
        'SD_test': X_test_2025[col].std(),
        't_stat': t_x.statistic,
        'p_value': t_x.pvalue
    })

tabla_medias_2025 = pd.DataFrame(resultados_2025)
tabla_medias_2025


C:\ProgramData\anaconda3\Lib\site-packages\scipy\stats\_axis_nan_policy.py:586: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  res = hypotest_fun_out(*samples, **kwds)


,Variable,Mean_train,Mean_test,SD_train,SD_test,t_stat,p_value
0,const,1.00,1.00,0.00,0.00,NaN,NaN
1,ano4,"2,025.00","2,025.00",0.00,0.00,NaN,NaN
2,ch06,37.66,38.14,22.12,22.09,-0.64,0.52
3,Sexo_Varón,0.50,0.48,0.50,0.50,0.82,0.41
4,nivel_ed_Primaria Incompleta (incluye educació...,0.14,0.14,0.34,0.35,-0.07,0.95
5,nivel_ed_Secundaria Completa,0.25,0.25,0.44,0.43,0.29,0.77
6,nivel_ed_Secundaria Incompleta,0.21,0.22,0.41,0.42,-0.66,0.51
7,nivel_ed_Sin instrucción,0.05,0.05,0.22,0.21,0.56,0.57
8,nivel_ed_Superior Universitaria Completa,0.13,0.12,0.34,0.33,0.96,0.34
9,nivel_ed_Superior Universitaria Incompleta,0.11,0.11,0.31,0.31,-0.19,0.85


### B. REGRESIÓN LOGÍSTICA

In [29]:
# === Modelo Logit ===
logit_model = sm.Logit(y_train_2025, X_train_2025)
logit_results = logit_model.fit()

logit_results.summary()


         Current function value: 0.441720
         Iterations: 35


C:\ProgramData\anaconda3\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                  pobre   No. Observations:                 2811
Model:                          Logit   Df Residuals:                     2786
Method:                           MLE   Df Model:                           24
Date:                Fri, 14 Nov 2025   Pseudo R-squ.:                  0.1153
Time:                        22:18:16   Log-Likelihood:                -1241.7
converged:                      False   LL-Null:                       -1403.6
Covariance Type:            nonrobust   LLR p-value:                 2.619e-54
=============================================================================================================================
                                                                coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------------------
const                                                     -4.938e-07        nan        nan        nan         nan         nan
ano4                                                          0.0004    306.617   1.35e-06      1.000    -600.957     600.958
ch06                                                         -0.0173      0.005     -3.484      0.000      -0.027      -0.008
Sexo_Varón                                                   -0.0133      0.106     -0.125      0.900      -0.221       0.195
nivel_ed_Primaria Incompleta (incluye educación especial)     0.0081      0.240      0.034      0.973      -0.463       0.479
nivel_ed_Secundaria Completa                                 -0.4956      0.207     -2.389      0.017      -0.902      -0.089
nivel_ed_Secundaria Incompleta                                0.0808      0.209      0.386      0.699      -0.329       0.491
nivel_ed_Sin instrucción                                      0.5523      0.541      1.021      0.307      -0.508       1.612
nivel_ed_Superior Universitaria Completa                     -2.2856      0.399     -5.722      0.000      -3.069      -1.503
nivel_ed_Superior Universitaria Incompleta                   -0.5160      0.251     -2.052      0.040      -1.009      -0.023
estado_Inactivo                                               0.1569   1.54e+06   1.02e-07      1.000   -3.02e+06    3.02e+06
estado_Menor de 10 años                                       0.2051   1.51e+06   1.36e-07      1.000   -2.96e+06    2.96e+06
estado_Ocupado                                               -0.7380        nan        nan        nan         nan         nan
cat_ocup_No ocupado                                          -0.9951      0.568     -1.751      0.080      -2.109       0.119
cat_ocup_Obrero o empleado                                   -0.4766      0.206     -2.308      0.021      -0.881      -0.072
cat_ocup_Patrón                                              -0.5902      0.773     -0.764      0.445      -2.104       0.924
cat_ocup_Trabajador familiar sin remuneración               -13.6712   1357.994     -0.010      0.992   -2675.290    2647.947
cat_inac_Desocupado                                           0.3760   1.54e+06   2.44e-07      1.000   -3.02e+06    3.02e+06
cat_inac_Discapacitado                                        0.6841      0.417      1.640      0.101      -0.134       1.502
cat_inac_Estudiante                                          -0.6144      0.402     -1.529      0.126      -1.402       0.173
cat_inac_Jubilado/pensionado                                 -1.1006      0.259     -4.248      0.000      -1.609      -0.593
cat_inac_Menor de 6 años                                     -1.4221      0.603     -2.359      0.018      -2.604      -0.240
cat_inac_Ocupado                                             -0.7380        nan        nan 

In [30]:
np.exp(logit_results.params)

const                                                       1.00
ano4                                                        1.00
ch06                                                        0.98
Sexo_Varón                                                  0.99
nivel_ed_Primaria Incompleta (incluye educación especial)   1.01
nivel_ed_Secundaria Completa                                0.61
nivel_ed_Secundaria Incompleta                              1.08
nivel_ed_Sin instrucción                                    1.74
nivel_ed_Superior Universitaria Completa                    0.10
nivel_ed_Superior Universitaria Incompleta                  0.60
estado_Inactivo                                             1.17
estado_Menor de 10 años                                     1.23
estado_Ocupado                                              0.48
cat_ocup_No ocupado                                         0.37
cat_ocup_Obrero o empleado                                  0.62
cat_ocup_Patrón          